# 10 - Matching

## Que fait la régression après tout ?

Comme nous l'avons vu jusqu'à présent, la régression fait un travail incroyable pour contrôler les variables supplémentaires lorsqu'on effectue une comparaison test vs contrôle. Si nous avons l'indépendance, $(Y_0, Y_1)\perp T | X$, alors la régression peut identifier l'ATE en contrôlant pour X. La manière dont la régression fait cela est un peu magique. Pour en avoir une intuition, souvenons-nous du cas où toutes les variables X sont des variables fictives (dummy variables). Si c'est le cas, la régression partitionne les données en cellules fictives et calcule la différence moyenne entre le test et le contrôle. Cette différence de moyennes maintient les X constants, puisque nous le faisons dans une cellule fixe de X fictif. C'est comme si nous faisions $E[Y|T=1] - E[Y|T=0] | X=x$, où $x$ est une cellule fictive (toutes les variables fictives définies à 1, par exemple). La régression combine ensuite les estimations dans chacune des cellules pour produire un ATE final. Elle le fait en appliquant des poids aux cellules proportionnellement à la variance du traitement dans ce groupe.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf

import graphviz as gr

%matplotlib inline

style.use("fivethirtyeight")

Pour donner un exemple, supposons que j'essaie d'estimer l'effet d'un médicament et que j'ai 6 hommes et 4 femmes. Ma variable de réponse est le nombre de jours d'hospitalisation et j'espère que mon médicament pourra réduire ce nombre. Chez les hommes, l'effet causal véritable est de -3, donc le médicament réduit la durée du séjour de 3 jours. Chez les femmes, il est de -2. Pour rendre les choses plus intéressantes, les hommes sont beaucoup plus touchés par cette maladie et restent plus longtemps à l'hôpital. Ils reçoivent également beaucoup plus le médicament. Un seul des 6 hommes ne reçoit pas le médicament. D'autre part, les femmes sont plus résistantes à cette maladie, donc elles restent moins longtemps à l'hôpital. 50 % des femmes reçoivent le médicament.

In [2]:
drug_example = pd.DataFrame(dict(
    sex= ["M","M","M","M","M","M", "W","W","W","W"],
    drug=[1,1,1,1,1,0,  1,0,1,0],
    days=[5,5,5,5,5,8,  2,4,2,4]
))

Notez qu'une simple comparaison entre le traitement et le contrôle donne un effet biaisé négativement, c'est-à-dire que le médicament semble moins efficace qu'il ne l'est réellement. Cela est attendu, puisque nous avons omis le facteur de confusion lié au sexe. Dans ce cas, l'ATE estimé est plus petit que le véritable ATE parce que les hommes reçoivent davantage le médicament et sont plus affectés par la maladie.

In [3]:
drug_example.query("drug==1")["days"].mean() - drug_example.query("drug==0")["days"].mean()

-1.1904761904761898

Puisque l'effet réel pour les hommes est de -3 et l'effet réel pour les femmes est de -2, l'ATE devrait être

$$
ATE = \dfrac{(-3*6) + (-2*4)}{10} = -2.6
$$

Cette estimation est réalisée en 1) partitionnant les données en cellules de confusion, dans ce cas, hommes et femmes, 2) estimant l'effet dans chaque cellule et 3) combinant les estimations avec une moyenne pondérée, où le poids est la taille de l'échantillon de la cellule ou du groupe de covariables. Si nous avions exactement le même nombre d'hommes et de femmes dans les données, l'estimation de l'ATE serait exactement au milieu de l'ATE des deux groupes, soit -2,5. Étant donné qu'il y a plus d'hommes que de femmes dans notre ensemble de données, l'estimation de l'ATE est un peu plus proche de l'ATE des hommes. Cela s'appelle une estimation non paramétrique, puisqu'elle ne repose sur aucune hypothèse concernant la façon dont les données ont été générées.

Si nous contrôlons le sexe en utilisant la régression, nous ajouterons l'hypothèse de linéarité. La régression partitionnera également les données en hommes et femmes et estimera l'effet sur ces deux groupes. Jusqu'ici, tout va bien. Cependant, lorsqu'il s'agit de combiner l'effet sur chaque groupe, elle ne les pèse pas par la taille de l'échantillon. Au lieu de cela, la régression utilise des poids proportionnels à la variance du traitement dans ce groupe. Dans notre cas, la variance du traitement chez les hommes est plus petite que chez les femmes, puisque seul un homme est dans le groupe de contrôle. Pour être exact, la variance de T pour les hommes est de $0.139 = 1/6*(1 - 1/6)$ et pour les femmes est de $0.25 = 2/4*(1 - 2/4)$. Ainsi, la régression donnera un poids plus élevé aux femmes dans notre exemple et l'ATE sera un peu plus proche de l'ATE des femmes de -2.

In [4]:
smf.ols('days ~ drug + C(sex)', data=drug_example).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,7.5455,0.188,40.093,0.000,7.100,7.990
C(sex)[T.W],-3.3182,0.176,-18.849,0.000,-3.734,-2.902
drug,-2.4545,0.188,-13.042,0.000,-2.900,-2.010


Ce résultat est plus intuitif avec des variables fictives, mais, d'une manière étrange, la régression maintient également les variables continues constantes tout en estimant l'effet. De plus, avec des variables continues, l'ATE pointera dans la direction où les covariables ont plus de variance.

Nous avons donc vu que la régression a ses particularités. Elle est linéaire, paramétrique, apprécie les caractéristiques à forte variance... Cela peut être bon ou mauvais, selon le contexte. C'est pourquoi il est important d'être conscient des autres techniques que nous pouvons utiliser pour contrôler les facteurs de confusion. Non seulement elles constituent un outil supplémentaire dans votre ceinture à outils causale, mais comprendre différentes manières de traiter les facteurs de confusion élargit notre compréhension du problème. Pour cette raison, je vous présente maintenant l'**Estimateur de Subclassification !**

## L'Estimateur de Subclassification

![img](./data/img/matching/explain.png)

S'il y a un effet causal que nous voulons estimer, comme l'effet de la formation professionnelle sur les revenus, **et** que le traitement n'est pas attribué au hasard, nous devons faire attention aux facteurs de confusion. Il se pourrait que seules les personnes plus motivées suivent la formation et qu'elles auraient des revenus plus élevés indépendamment de la formation. Nous devons estimer l'effet du programme de formation au sein de petits groupes d'individus qui sont à peu près au même niveau de motivation et de tout autre facteur de confusion que nous pourrions avoir.

Plus généralement, s'il y a un effet causal que nous voulons estimer, mais qu'il est difficile de le faire à cause de la confusion de certaines variables X, ce que nous devons faire est de faire la comparaison traitement vs contrôle au sein de petits groupes où X est le même. Si nous avons l'indépendance conditionnelle $(Y_0, Y_1)\perp T | X$, alors nous pouvons écrire l'ATE comme suit.

$$
ATE = \int(E[Y|X, T=1] - E[Y|X, T=0])dP(x)
$$

Ce que fait cette intégrale, c'est qu'elle parcourt tout l'espace de la distribution des caractéristiques X, calcule la différence de moyennes pour tous ces petits espaces et combine le tout en l'ATE. Une autre façon de voir cela est de penser à un ensemble discret de caractéristiques. Dans ce cas, nous pouvons dire que les caractéristiques X prennent K différentes cellules $\{X_1, X_2, ..., X_k\}$ et ce que nous faisons est de calculer l'effet du traitement dans chaque cellule et de les combiner dans l'ATE. Dans ce cas discret, en convertissant l'intégrale en une somme, nous pouvons dériver l'estimateur de subclassification.

$$
\hat{ATE} = \sum^K_{k=1}(\bar{Y}_{k1} - \bar{Y}_{k0}) * \dfrac{N_k}{N}
$$

où la barre représente la moyenne du résultat pour les traités, $Y_{k1}$, et les non-traités, $Y_{k0}$, dans la cellule k et $N_{k}$ est le nombre d'observations dans cette même cellule. Comme vous pouvez le voir, nous calculons un ATE local pour chaque cellule et les combinons en utilisant une moyenne pondérée, où les poids sont la taille de l'échantillon de la cellule. Dans notre exemple de médicament ci-dessus, ce serait la première estimation, qui nous a donné -2,6.

## Estimateur de Matching

![img](./data/img/matching/its-a-match.png)

L'estimateur de subclassification n'est pas beaucoup utilisé en pratique (nous verrons pourquoi sous peu, c'est à cause du fléau de la dimensionnalité), mais il nous donne une bonne intuition de ce que devrait faire un estimateur d'inférence causale, comment il devrait contrôler les facteurs de confusion. Cela nous permet d'explorer d'autres types d'estimateurs, comme l'estimateur de matching.

L'idée est très similaire. Étant donné qu'un certain type de facteur de confusion X fait en sorte que les traités et les non-traités ne sont pas initialement comparables, je peux les rendre comparables en **appariant chaque unité traitée avec une unité non traitée similaire**. C'est comme si je trouvais un jumeau non traité pour chaque unité traitée. En faisant de telles comparaisons, les traités et les non-traités redeviennent comparables.

À titre d'exemple, supposons que nous essayons d'estimer l'effet d'un programme de formation sur les revenus. Voici à quoi ressemblent les stagiaires.

In [5]:
trainee = pd.read_csv("./data/trainees.csv")
trainee.query("trainees==1")

,unit,trainees,age,earnings
0,1,1,28,17700
1,2,1,34,10200
2,3,1,29,14400
3,4,1,25,20800
4,5,1,29,6100
5,6,1,23,28600
6,7,1,33,21900
7,8,1,27,28800
8,9,1,31,20300
9,10,1,26,28100


Et les non-stagiaires:

In [6]:
trainee.query("trainees==0")

,unit,trainees,age,earnings
19,20,0,43,20900
20,21,0,50,31000
21,22,0,30,21000
22,23,0,27,9300
23,24,0,54,41100
24,25,0,48,29800
25,26,0,39,42000
26,27,0,28,8800
27,28,0,24,25500
28,29,0,33,15500


Si je fais une simple comparaison des moyennes, nous constatons que les stagiaires gagnent moins d'argent que ceux qui n'ont pas suivi le programme.

In [7]:
trainee.query("trainees==1")["earnings"].mean() - trainee.query("trainees==0")["earnings"].mean()

-4297.49373433584

Cependant, si nous regardons le tableau ci-dessus, nous remarquons que les stagiaires sont beaucoup plus jeunes que les non-stagiaires, ce qui indique que l'âge est probablement un facteur de confusion. Utilisons le matching sur l'âge pour essayer de corriger cela. Nous prendrons l'unité 1 parmi les traités et l'apparierons avec l'unité 27, car les deux ont 28 ans. L'unité 2 sera appariée avec l'unité 34, l'unité 3 avec l'unité 37, l'unité 4 sera appariée avec l'unité 35... En ce qui concerne l'unité 5, nous devons trouver quelqu'un âgé de 29 ans parmi les non-traités, mais c'est l'unité 37, qui est déjà appariée. Cela ne pose pas de problème, car nous pouvons utiliser la même unité plusieurs fois. Si plusieurs unités sont compatibles, nous pouvons choisir au hasard parmi elles.

Voici à quoi ressemble l'ensemble de données appariées pour les 7 premières unités.

In [8]:
# make dataset where no one has the same age
unique_on_age = (trainee
                 .query("trainees==0")
                 .drop_duplicates("age"))

matches = (trainee
           .query("trainees==1")
           .merge(unique_on_age, on="age", how="left", suffixes=("_t_1", "_t_0"))
           .assign(t1_minuts_t0 = lambda d: d["earnings_t_1"] - d["earnings_t_0"]))

matches.head(7)

,unit_t_1,trainees_t_1,age,earnings_t_1,unit_t_0,trainees_t_0,earnings_t_0,t1_minuts_t0
0,1,1,28,17700,27,0,8800,8900
1,2,1,34,10200,34,0,24200,-14000
2,3,1,29,14400,37,0,6200,8200
3,4,1,25,20800,35,0,23300,-2500
4,5,1,29,6100,37,0,6200,-100
5,6,1,23,28600,40,0,9500,19100
6,7,1,33,21900,29,0,15500,6400


Remarquez comment la dernière colonne montre la différence de revenus entre l'unité traitée et son unité non traitée appariée. Si nous prenons la moyenne de cette dernière colonne, nous obtenons l'estimation de l'ATET tout en contrôlant l'âge. Remarquez comment l'estimation est maintenant très positive, comparée à celle précédente où nous avions utilisé une simple différence de moyennes.

In [9]:
matches["t1_minuts_t0"].mean()

2457.8947368421054

Mais ceci était un exemple très artificiel, juste pour introduire le matching. En réalité, nous avons généralement plus d'une caractéristique et les unités ne correspondent pas parfaitement. Dans ce cas, nous devons définir une mesure de proximité pour comparer à quel point les unités sont proches les unes des autres. Une métrique courante pour cela est la norme euclidienne $||X_i - X_j||$. Cette différence, cependant, n'est pas invariante à l'échelle des caractéristiques. Cela signifie que des caractéristiques comme l'âge, qui prennent des valeurs en dizaines, seront beaucoup moins importantes lors du calcul de cette norme par rapport à des caractéristiques comme le revenu, qui prennent des valeurs de l'ordre de centaines. Pour cette raison, avant d'appliquer la norme, nous devons mettre les caractéristiques à l'échelle pour qu'elles soient à peu près sur la même échelle.

Ayant défini une mesure de distance, nous pouvons maintenant définir le matching comme le plus proche voisin de cet échantillon que nous souhaitons apparier. En termes mathématiques, nous pouvons écrire l'estimateur de matching de la manière suivante :

$$
\hat{ATE} = \frac{1}{N} \sum^N_{i=1} (2T_i - 1)\big(Y_i - Y_{jm}(i)\big)
$$

Où $Y_{jm}(i)$ est l'échantillon du groupe de traitement opposé qui est le plus similaire à $Y_i$. Nous faisons ce $2T_i - 1$ pour apparier dans les deux sens : traités avec les contrôles et contrôles avec les traités.

Pour tester cet estimateur, considérons un exemple de médecine. Encore une fois, nous voulons trouver l'effet d'un médicament sur le nombre de jours jusqu'à la guérison. Malheureusement, cet effet est confondu par la gravité, le sexe et l'âge. Nous avons des raisons de croire que les patients ayant des conditions plus graves ont une probabilité plus élevée de recevoir le médicament.

In [10]:
med = pd.read_csv("./data/medicine_impact_recovery.csv")
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


Si nous examinons une simple différence de moyennes, $E[Y|T=1]-E[Y|T=0]$, nous constatons que le traitement prend, en moyenne, 16,9 jours de plus pour se rétablir que les non-traités. Cela est probablement dû à des facteurs de confusion, car nous ne nous attendons pas à ce que le médicament nuise au patient.

In [11]:
med.query("medication==1")["recovery"].mean() - med.query("medication==0")["recovery"].mean()

16.895799546498726

Pour corriger ce biais, nous allons contrôler les variables X en utilisant le matching. Tout d'abord, nous devons nous rappeler de mettre nos caractéristiques à l'échelle, sinon des caractéristiques comme l'âge auront plus d'importance que des caractéristiques comme la gravité lorsque nous calculerons la distance entre les points. Pour ce faire, nous pouvons standardiser les caractéristiques.

In [12]:
# scale features
X = ["severity", "age", "sex"]
y = "recovery"

med = med.assign(**{f: (med[f] - med[f].mean())/med[f].std() for f in X})
med.head()

,sex,age,severity,medication,recovery
0,-0.996980,0.280787,1.459800,1,31
1,1.002979,0.865375,1.502164,1,49
2,1.002979,-0.338749,0.057796,0,38
3,1.002979,0.399465,-0.512557,0,35
4,-0.996980,-0.610473,-0.911125,0,15


Maintenant, passons au matching lui-même. Au lieu de coder une fonction de matching, nous utiliserons l'algorithme des K plus proches voisins de [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html). Cet algorithme fait des prédictions en trouvant le point de données le plus proche dans un ensemble d'estimation ou d'entraînement.

Pour le matching, nous aurons besoin de 2 de ces modèles. L'un, `mt0`, stockera les points non traités et trouvera des correspondances parmi les non traités lorsqu'on le lui demandera. L'autre, `mt1`, stockera les points traités et trouvera des correspondances parmi les traités lorsqu'on le lui demandera. Après cette étape de fitting, nous pourrons utiliser ces modèles KNN pour faire des prédictions, qui seront nos correspondances.

In [13]:
from sklearn.neighbors import KNeighborsRegressor

treated = med.query("medication==1")
untreated = med.query("medication==0")

mt0 = KNeighborsRegressor(n_neighbors=1).fit(untreated[X], untreated[y])
mt1 = KNeighborsRegressor(n_neighbors=1).fit(treated[X], treated[y])

predicted = pd.concat([
    # find matches for the treated looking at the untreated knn model
    treated.assign(match=mt0.predict(treated[X])),
    
    # find matches for the untreated looking at the treated knn model
    untreated.assign(match=mt1.predict(untreated[X]))
])

predicted.head()

,sex,age,severity,medication,recovery,match
0,-0.996980,0.280787,1.459800,1,31,39.0
1,1.002979,0.865375,1.502164,1,49,52.0
7,-0.996980,1.495134,1.268540,1,38,46.0
10,1.002979,-0.106534,0.545911,1,34,45.0
16,-0.996980,0.043034,1.428732,1,30,39.0


Avec les observations appariées, nous pouvons maintenant appliquer la formule de l'estimateur de matching

$
\hat{ATE} = \frac{1}{N} \sum^N_{i=1} (2T_i - 1)\big(Y_i - Y_{jm}(i)\big)
$

In [14]:
np.mean((2*predicted["medication"] - 1)*(predicted["recovery"] - predicted["match"]))

-0.9954

En utilisant ce type d'appariement, nous pouvons voir que l'effet du médicament n'est plus positif. Cela signifie qu'en contrôlant pour X, le médicament réduit le temps de récupération d'environ un jour, en moyenne. C'est déjà une amélioration considérable par rapport à l'estimation biaisée qui prédisait une augmentation de 16,9 jours du temps de récupération.

Cependant, nous pouvons encore faire mieux.

## Biais de Matching

Il s'avère que l'estimateur de matching tel que nous l'avons conçu ci-dessus est biaisé. Pour comprendre cela, considérons l'estimateur ATET (Average Treatment Effect on the Treated), au lieu de l'ATE, simplement parce qu'il est plus simple à écrire. L'intuition s'appliquera également à l'ATE.

$$
\hat{ATET} = \frac{1}{N_1}\sum (Y_i - Y_j(i))
$$

où $N_1$ est le nombre d'individus traités et $Y_j(i)$ est la correspondance non traitée de l'unité traitée i. Pour vérifier le biais, ce que nous faisons, c'est espérer pouvoir appliquer le Théorème Central Limite afin que ce qui suit converge vers une distribution normale avec une moyenne de zéro.

$$
\sqrt{N_1}(\hat{ATET} - ATET)
$$

Cependant, cela n'arrive pas toujours. Si nous définissons la moyenne des résultats pour les non-traités en fonction de X, $\mu_0(x)=E[Y|X=x, T=0]$, nous aurons que (entre parenthèses, j'ai omis la démonstration car elle dépasse un peu le sujet ici).

$$
E[\sqrt{N_1}(\hat{ATET} - ATET)] = E[\sqrt{N_1}(\mu_0(X_i) - \mu_0(X_j(i)))]
$$

Maintenant, $\mu_0(X_i) - \mu_0(X_j(i))$ n'est pas si simple à comprendre, alors examinons-le plus attentivement. $\mu_0(X_i)$ est la valeur de Y d'une unité traitée $i$ si elle n'avait pas été traitée. C'est donc le résultat contrefactuel $Y_0$ pour l'unité i. $\mu_0(X_j(i))$ est le résultat de l'unité non traitée $j$ qui est l'appariement de l'unité $i$. C'est donc également le $Y_0$, mais pour l'unité $j$ maintenant. Cette fois-ci, c'est un résultat factuel, car $j$ est dans le groupe non traité. Comme $j$ et $i$ sont seulement similaires, mais pas identiques, cela ne sera probablement pas nul. En d'autres termes, $X_i \approx X_j$. Donc, $Y_{0i} \approx Y_{0j}$.

À mesure que nous augmentons la taille de l'échantillon, il y aura plus d'unités à apparier, donc la différence entre l'unité $i$ et son appariement $j$ diminuera également. Mais cette différence converge lentement vers zéro. En conséquence, $E[\sqrt{N_1}(\mu_0(X_i) - \mu_0(X_j(i)))]$ peut ne pas converger vers zéro, car le $\sqrt{N_1}$ croît plus rapidement que $(\mu_0(X_i) - \mu_0(X_j(i)))$ ne diminue.

Le biais apparaît lorsque les écarts de matching sont importants. Heureusement, nous savons comment le corriger. Chaque observation contribue $(\mu_0(X_i) - \mu_0(X_j(i)))$ au biais, donc tout ce que nous avons à faire est de soustraire cette quantité de chaque comparaison de matching dans notre estimateur. Pour ce faire, nous pouvons remplacer $\mu_0(X_j(i))$ par une sorte d'estimation de cette quantité $\hat{\mu_0}(X_j(i))$, qui peut être obtenue avec des modèles comme la régression linéaire. Cela met à jour l'estimateur ATET avec l'équation suivante :

$$
\hat{ATET} = \frac{1}{N_1}\sum \big((Y_i - Y_{j(i)}) - (\hat{\mu_0}(X_i) - \hat{\mu_0}(X_{j(i)}))\big)
$$

où $\hat{\mu_0}(x)$ est une estimation de $E[Y|X, T=0]$, comme une régression linéaire ajustée uniquement sur l'échantillon non traité.

In [15]:
from sklearn.linear_model import LinearRegression

# fit the linear regression model to estimate mu_0(x)
ols0 = LinearRegression().fit(untreated[X], untreated[y])
ols1 = LinearRegression().fit(treated[X], treated[y])

# find the units that match to the treated
treated_match_index = mt0.kneighbors(treated[X], n_neighbors=1)[1].ravel()

# find the units that match to the untreatd
untreated_match_index = mt1.kneighbors(untreated[X], n_neighbors=1)[1].ravel()

predicted = pd.concat([
    (treated
     # find the Y match on the other group
     .assign(match=mt0.predict(treated[X])) 
     
     # build the bias correction term
     .assign(bias_correct=ols0.predict(treated[X]) - ols0.predict(untreated.iloc[treated_match_index][X]))),
    (untreated
     .assign(match=mt1.predict(untreated[X]))
     .assign(bias_correct=ols1.predict(untreated[X]) - ols1.predict(treated.iloc[untreated_match_index][X])))
])

predicted.head()

,sex,age,severity,medication,recovery,match,bias_correct
0,-0.996980,0.280787,1.459800,1,31,39.0,4.404034
1,1.002979,0.865375,1.502164,1,49,52.0,12.915348
7,-0.996980,1.495134,1.268540,1,38,46.0,1.871428
10,1.002979,-0.106534,0.545911,1,34,45.0,-0.496970
16,-0.996980,0.043034,1.428732,1,30,39.0,2.610159


Une question immédiate qui se pose est : cela ne va-t-il pas à l'encontre de l'objectif du matching ? Si je dois de toute façon effectuer une régression linéaire, pourquoi ne pas utiliser uniquement cela, au lieu de ce modèle compliqué ? C'est un point valable, donc je devrais prendre un moment pour y répondre.

![img](./data/img/matching/ubiquitous-ols.png)

Tout d'abord, cette régression linéaire que nous ajustons ne fait pas d'extrapolation sur la dimension du traitement pour obtenir l'effet du traitement. Son objectif est plutôt de corriger le biais. La régression linéaire ici est locale, dans le sens où elle ne cherche pas à voir comment les traités se comporteraient s'ils ressemblaient aux non-traités. Elle ne fait aucune de ces extrapolations. Cela est laissé à la partie matching. L'essentiel de l'estimateur reste la composante de matching. Ce que je veux dire ici, c'est que la régression linéaire (OLS) est secondaire dans cet estimateur.

Le deuxième point est que le matching est un estimateur non paramétrique. Il ne suppose pas de linéarité ou de modèle paramétrique quelconque. En tant que tel, il est plus flexible que la régression linéaire et peut fonctionner dans des situations où la régression linéaire ne le pourrait pas, notamment lorsque la non-linéarité est très forte.

Cela signifie-t-il que vous devriez utiliser uniquement le matching ? Eh bien, c'est une question difficile. Alberto Abadie affirme que oui, vous devriez. C'est plus flexible et, une fois que vous avez le code, tout aussi simple à exécuter. Je ne suis pas entièrement convaincu par cela. Par exemple, Abadie a passé beaucoup de temps à étudier et développer l'estimateur (oui, il est l'un des scientifiques qui ont contribué à faire du matching ce qu'il est), il est donc évidemment personnellement investi dans la méthode. Deuxièmement, il y a quelque chose dans la simplicité de la régression linéaire que vous ne voyez pas dans le matching. Les mathématiques de la dérivée partielle consistant à "garder tout le reste constant" sont beaucoup plus faciles à comprendre avec la régression linéaire qu'avec le matching. Mais c'est juste ma préférence. Pour être honnête, il n'y a pas de réponse claire à cette question. Quoi qu'il en soit, revenons à notre exemple.

Avec la formule de correction du biais, j'obtiens l'estimation suivante de l'ATE.

In [16]:
np.mean((2*predicted["medication"] - 1)*((predicted["recovery"] - predicted["match"])-predicted["bias_correct"]))

-7.36266090614141

Bien sûr, nous devons également placer un intervalle de confiance autour de cette mesure, mais assez de théorie mathématique pour l’instant. En pratique, nous pouvons simplement utiliser le code de quelqu’un d’autre et importer un estimateur de matching. En voici un de la bibliothèque  [causalinference](https://github.com/laurencium/causalinference). 

In [17]:
from causalinference import CausalModel

cm = CausalModel(
    Y=med["recovery"].values, 
    D=med["medication"].values, 
    X=med[["severity", "age", "sex"]].values
)

cm.est_via_matching(matches=1, bias_adj=True)

print(cm.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -7.709      0.609    -12.649      0.000     -8.903     -6.514
           ATC     -6.665      0.246    -27.047      0.000     -7.148     -6.182
           ATT     -9.679      1.693     -5.717      0.000    -12.997     -6.361



Enfin, nous pouvons dire avec confiance que notre médicament réduit effectivement le temps passé à l'hôpital. L'estimation ATE est juste un peu plus basse que la mienne, en raison de la différence dans le dénouement des égalités des correspondances de l'implémentation knn de sklearn et du package Python causalinference.

Avant de clore ce sujet, je voulais juste aborder un peu plus la cause du biais dans le matching. Nous avons vu que le matching est biaisé lorsque l'unité et sa correspondance ne sont pas si similaires. Mais qu'est-ce qui les rend si différents ?

## Le Fléau de la Dimensionalité

Il s'avère que la réponse est assez simple et intuitive. Il est facile de trouver des personnes qui correspondent sur quelques caractéristiques, comme le sexe. Mais si nous ajoutons plus de caractéristiques, comme l'âge, le revenu, la ville de naissance, etc., il devient de plus en plus difficile de trouver des correspondances. En termes plus généraux, plus nous avons de caractéristiques, plus la distance entre les unités et leurs correspondances sera élevée.

Cela ne nuit pas seulement à l'estimateur de matching. Cela se rattache à l'estimateur de sous-classification que nous avons vu plus tôt. Au début, dans cet exemple de médicament inventé avec des hommes et des femmes, il était assez facile de construire l'estimateur de sous-classification. C'était parce que nous n'avions que 2 cellules : homme et femme. Mais que se passerait-il si nous en avions plus ? Supposons que nous ayons 2 caractéristiques continues comme l'âge et le revenu et que nous réussissions à les discrétiser en 5 catégories chacune. Cela nous donnerait 25 cellules, ou $5^2$. Et si nous avions 10 covariables avec 3 catégories chacune ? Cela ne semble pas beaucoup, n'est-ce pas ? Eh bien, cela nous donnerait 59049 cellules, ou $3^{10}$. Il est facile de voir comment cela peut rapidement devenir disproportionné. C'est un phénomène omniprésent dans toute la science des données, appelé **Le Fléau de la Dimensionalité** !!!

![img](./data/img/curse-of-dimensionality.jpg)
Source de l'image : https://deepai.org/machine-learning-glossary-and-terms/curse-of-dimensionality

Malgré son nom effrayant et prétentieux, cela signifie simplement que le nombre de points de données nécessaires pour remplir un espace de caractéristiques augmente de façon exponentielle avec le nombre de caractéristiques, ou dimensions. Donc, s'il faut X points de données pour remplir l'espace de, disons, 3 espaces de caractéristiques, il en faut exponentiellement plus pour remplir l'espace de 4 caractéristiques.

Dans le contexte de l'estimateur de sous-classification, le fléau de la dimensionalité signifie qu'il souffrira si nous avons beaucoup de caractéristiques. Beaucoup de caractéristiques impliquent de multiples cellules en X. S'il y a plusieurs cellules, certaines d'entre elles auront très peu de données. Certaines pourraient même avoir seulement des traitements ou seulement des contrôles, il ne sera donc pas possible d'estimer l'ATE là-bas, ce qui casserait notre estimateur. Dans le contexte du matching, cela signifie que l'espace des caractéristiques sera très clairsemé et que les unités seront très éloignées les unes des autres. Cela augmentera la distance entre les correspondances et causera des problèmes de biais.

En ce qui concerne la régression linéaire, elle gère en fait très bien ce problème. Ce qu'elle fait, c'est projeter toutes les caractéristiques X sur une seule, la dimension Y. Elle compare ensuite le traitement et le contrôle sur cette projection. Donc, d'une certaine manière, la régression linéaire effectue une sorte de réduction de dimension pour estimer l'ATE. C'est assez élégant.

La plupart des modèles causaux ont également une méthode pour traiter le fléau de la dimensionalité. Je ne vais pas me répéter, mais c'est quelque chose dont vous devriez vous souvenir lorsque vous les examinez. Par exemple, lorsque nous traitons des scores de propension dans la section suivante, essayez de voir comment ils résolvent ce problème.

## Idées Clés

Nous avons commencé cette section en comprenant ce que fait la régression linéaire et comment elle peut nous aider à identifier les relations causales. À savoir, nous avons compris que la régression peut être vue comme la partition du jeu de données en cellules, le calcul de l'ATE dans chaque cellule, puis la combinaison de l'ATE des cellules en un seul ATE pour l'ensemble du jeu de données.

À partir de là, nous avons dérivé un estimateur d'inférence causale très général avec sous-classification. Nous avons vu comment cet estimateur n'est pas très utile en pratique, mais il nous a donné des idées intéressantes sur la manière de résoudre le problème de l'estimation de l'inférence causale. Cela nous a donné l'opportunité de parler de l'estimateur de matching.

Le matching contrôle les variables confondantes en regardant chaque unité traitée et en trouvant une paire non traitée qui lui est très similaire et de même pour les unités non traitées. Nous avons vu comment mettre en œuvre cette méthode en utilisant l'algorithme KNN et aussi comment la débiasser en utilisant la régression. Enfin, nous avons discuté de la différence entre le matching et la régression linéaire. Nous avons vu comment le matching est un estimateur non paramétrique qui ne repose pas sur la linéarité comme le fait la régression linéaire.

Enfin, nous avons abordé le problème des jeux de données à haute dimension et nous avons vu comment les méthodes d'inférence causale peuvent en souffrir.

## Références

J'aime penser à ce livre entier comme un hommage à Joshua Angrist, Alberto Abadie et Christopher Walters pour leur incroyable cours d'économétrie. La plupart des idées ici sont tirées de leurs cours à l'American Economic Association. Les regarder est ce qui me maintient sain d'esprit pendant cette année difficile de 2020.
* [Cross-Section Econometrics](https://www.aeaweb.org/conference/cont-ed/2017-webcasts)
* [Mastering Mostly Harmless Econometrics](https://www.aeaweb.org/conference/cont-ed/2020-webcasts)

Je tiens également à référencer les livres incroyables d'Angrist. Ils m'ont montré que l'économétrie, ou 'Metrics' comme ils l'appellent, n'est pas seulement extrêmement utile mais aussi profondément amusante.

* [Mostly Harmless Econometrics](https://www.mostlyharmlesseconometrics.com/)
* [Mastering 'Metrics](https://www.masteringmetrics.com/)

Ma dernière référence est le livre de Miguel Hernan et Jamie Robins. Il a été mon compagnon fidèle dans les questions causales les plus épineuses que j'ai dû résoudre.

* [Causal Inference Book](https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/)

Les données utilisées ici proviennent d'une étude de Alpert, William T., Kenneth A. Couch, et Oskar R. Harmon. 2016. ["A Randomized Assessment of Online Learning"](https://www.aeaweb.org/articles?id=10.1257/aer.p20161057). American Economic Review, 106 (5): 378-82.

![img](./data/img/poetry.png)

## Contribuer

*L'Inférence Causale pour les Courageux et les Vrais* est un matériel open-source sur l'inférence causale, la statistique de la science. Son objectif est d'être accessible monétairement et intellectuellement. Il utilise uniquement des logiciels gratuits basés sur Python.
Si vous avez trouvé ce livre précieux et souhaitez le soutenir, veuillez vous rendre sur [Patreon](https://www.patreon.com/causal_inference_for_the_brave_and_true). Si vous n'êtes pas prêt à contribuer financièrement, vous pouvez également aider en corrigeant les fautes de frappe, en suggérant des modifications ou en donnant votre avis sur les passages que vous n'avez pas compris. Rendez-vous sur le repo du livre et [ouvrez une issue](https://github.com/matheusfacure/python-causality-handbook/issues). Enfin, si vous avez aimé ce contenu, veuillez le partager avec d'autres personnes qui pourraient le trouver utile et lui donner une [étoile sur GitHub](https://github.com/matheusfacure/python-causality-handbook/stargazers).